<a href="https://colab.research.google.com/github/alxanderpierre/Unit-2-project/blob/master/Unit_2_Final_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction 

The city government of Los Angeles makes a portion of their city income from city parking citations. However in the recent months Mayor Gardenhire has been forced to cut 100 jobs from 7 city ran parking agencies, and he is planning to cut more . Thes jobs were cut impart to poor city budgeting, but mainly due to the new civil roadway projects going on downtown.  

The Mayor is entering an election year and does not want to increase taxes on the public to pay for the much needed roadway project, based on the fear of not being re-elected. Instead he has chosen to let go of several city jobs that will likely go unnoticed by the voting public. Parking citation workers are one of these jobs. 

Here is where we come in. As a data scientist for the city’s board of trustee’s office you have been asked to help with this issue. 

**Our goal is to create a model for these 7 agencies that will predict the most profitable areas based off time and price of ticket to protol in hopes to increase the citation revenue. Remember these agencies are short handed due to the job cuts, so the accuracy has to be high due to their inability to cover a large area. People’s job security is on the line here so lets make them a great model** . 


# Import and setup 

In [0]:
import pandas as pd
import numpy as np
import sys
!{sys.executable} -m pip install category_encoders
import sys
!{sys.executable} -m pip install xgboost
import category_encoders as ce
from xgboost import XGBClassifier
import sys
!{sys.executable} -m pip install eli5
from eli5.sklearn import PermutationImportance
import category_encoders as ce
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import sys
!{sys.executable} -m pip install pdpbox
%matplotlib inline
import matplotlib.pyplot as plt
from pdpbox import pdp
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix

In [0]:
import sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install packages in Colab
    !pip install category_encoders==2.0.0
    !pip install eli5==0.10.1
    !pip install pandas-profiling==2.3.0
    !pip install pdpbox==0.2.0
    !pip install plotly==4.1.1
    !pip install shap==0.30.0

# Load and Clean our data.

In [0]:
from google.colab import files 
uploaded = files.upload()

In [0]:
df = pd.read_csv('parking-citations.csv')

In [0]:
df_time = df['Issue time']
df_time.value_counts(ascending=False, bins=6)# even tho it my start the time at 787 we know that that just how 
# python binned the data. we know that the mins stop at 59 in the dataframe. 
# next lets put these bins back in the dataframe so that they can be cata and be easier to create a cata model 

In [0]:
df_time = pd.DataFrame(df_time)
df_time['time_bins'] = pd.cut(x=df['Issue time'],
                              bins=[-2,400,800,1200,1600,2000,2400],labels= ['12:00am - 4:00am','4:00am - 8:00am', '8:00am - 12:00pm', '12:00pm - 4:00pm',
                                       '4:00pm - 8:00pm', '8:00pm - Midnight'])

In [0]:
df['Time Bins'] = df_time['time_bins'] 

# Drop unnessary columns and rows

In [0]:
df_new = df.drop(columns=['Plate Expiry Date','RP State Plate','Violation code','Violation Description','Ticket number',
                          'Marked Time', 'RP State Plate','VIN','Violation code'])

In [0]:
df_new = df_new.drop(index=53557)

In [0]:
df_new.head()

In [0]:
df_new['Fine amount'] = df_new['Fine amount'].astype(str)
df_new['Agency'] = df_new['Agency'].astype(str)


In [0]:
df['Issue Date'].sort_values()

In [0]:
df.shape, df_new.shape

In [0]:
df['Fine amount'].value_counts()

In [0]:
df_new['Fine amount'].value_counts()

These columns were listed as floats/ int however they are in reality catagorical var and should be repersented as such 



# Ticket Revenue

In [0]:
ticket = pd.DataFrame(df_new['Fine amount'].value_counts())

In [0]:
tickets = {'ticket amounts':[73,63,68,93,25,58,163,363,293,78,250,1000,500,53,50,143],
           'ticket counts': [20177,8814,7388,6941,4879,4064,669,306,105,85,63,30,24,8,3,1],
           'ticket revenue':[1468541,555282,502384,645513,121975,235712,109047,111078,30765,6630,15750,30000,12000,424,429,50]}
tickets= pd.DataFrame(data=tickets)

In [0]:
tickets.sum(axis =0, skipna = True)

In [0]:
tickets = tickets.sort_values(by='ticket revenue',ascending=0)
tickets

In [0]:
highlight_tickets = tickets.head(8).drop(index=4)

In [0]:
highlight_tickets = highlight_tickets.drop(index=6)


In [0]:
highlight_tickets

To optimize for the most profitable tickets we have been told by our superiors  to focus on tickets that are 120k and up. The 25 dollar tickets are sitting at 121,975 total and the $363 are sitting at 111078 dollar total. 

Why this is important to highligh? 
Because it took 4879 25 dollar citations to get 121K, and just 306 to get 111k for teh 363 dollar citations. 

Remember man power low and we are having to work efficiently. Therefore we are going to set our model to target the 363 dollar tickets over the 25 dollars due to it taking 15X more man power to generate nearly the same amount of revenue. 

In [0]:
df_new = df_new.rename(columns={'Fine amount':'fine_amount'})

In [0]:
df_new.shape

In [0]:
df_new = df_new[df_new.fine_amount != '50.0'] 

In [0]:
df_new = df_new[df_new.fine_amount != '143.0']

In [0]:
df_new = df_new[df_new.fine_amount != '53.0']

In [0]:
df_new = df_new[df_new.fine_amount != '78.0']

In [0]:
df_new = df_new[df_new.fine_amount != '500.0']

In [0]:
df_new = df_new[df_new.fine_amount != '250.0']

In [0]:
df_new = df_new[df_new.fine_amount != '293.0']

In [0]:
df_new = df_new[df_new.fine_amount != '1000.0']

In [0]:
df_new = df_new[df_new.fine_amount != '163.0']

In [0]:
df_new = df_new[df_new.fine_amount != '25.0']

In [0]:
df_new.shape

# Make X and y dataframes for train and test.
# The target the is Fine Amount column 

In [0]:
X = df_new.drop(columns=['fine_amount'])
y = df_new['fine_amount']
features = df_new.drop(columns=['fine_amount'])

In [0]:
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size=.20, random_state=42)
#Choose which observations you will use to train, validate, and test your model

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=.80, test_size=0.20, random_state=42)
X_train.shape, X_val.shape , y_train.shape, y_val.shape

This is a class imbalanced classification problem
can we predict the price of a parking ticket based off the presenced varables 


In [0]:
y_train.value_counts(normalize=True) # Ticket price precentage 

In [0]:
highlight_tickets

What i notice here is that tickets that 93.00 dollars are the most valueable ticket due to it low volume yet high return. Excellent item to focus on with our current man power problem. 

In [0]:
X_train.describe().T

Descriptive stats for the non-numberic features


In [0]:
X_train.describe(exclude= 'number').T.sort_values(by='unique')

Time Bins looks like it could be a highly predictive feature 


In [0]:
X_train['Time Bins'].value_counts()

In [0]:
X_train.isnull().sum().sort_values()

See if we can fit a quick baseline with Time bins as the feature 

# Acuracy Score and Pipeline fit 

In [0]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        n_jobs=-1,
    )
)
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))

In [0]:
print('Test Accuracy', pipeline.score(X_test, y_test))

# Permutation Importance and Feature Importance 

In [0]:
transformers = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

In [0]:
X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

model.fit(X_train_transformed, y_train)

In [0]:
model.score(X_val_transformed, y_val)

In [0]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    model,
    scoring='accuracy', 
    n_iter=2, 
    random_state=42
)


In [0]:
permuter.fit(X_val_transformed, y_val)
feature_names = X_val.columns.tolist()

eli5.show_weights(
    permuter, 
    top=None, # show permutation importances for all features
    feature_names=feature_names)

In [0]:
import pandas as pd

rf = pipeline.named_steps['randomforestclassifier']
importances = pd.Series(rf.feature_importances_, X_train.columns)

%matplotlib inline
import matplotlib.pyplot as plt
n=20
plt.figure(figsize=(10,n/2))
plt.title(f'Ranking Features')
importances.sort_values()[-n:].plot.barh(color='green');

In [0]:
example = df_new[['fine_amount', 'Time Bins', 'Agency','Issue time']]
example.fine_amount = example.fine_amount.astype(float)
example

In [0]:
import seaborn as sns
plt.figure(figsize=(10,5))
sns.barplot(x='Agency', y='fine_amount',data=example, color='grey');


# Shapley Plot 

In [0]:
import numpy as np 

In [0]:
features = ['Issue time', 'Meter Id',  'Agency','Latitude', 'Longitude', 'Make', 'Route', 'Time Bins']

In [0]:
preprocessor = ce.OrdinalEncoder()
X_train_transformed = preprocessor.fit_transform(X_train[features])

In [0]:
 data_for_prediction = X_train_transformed.loc[0]

In [0]:
X_train[features].loc[0]

In [0]:
data_for_prediction

In [0]:

import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(data_for_prediction)

shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0], data_for_prediction
)

# Map

In [0]:
import plotly.express as px
px.scatter(df_new, x='Longitude', y='Latitude', color='fine_amount', opacity=.7,marginal_y="rug", marginal_x="histogram")

In [0]:
px.scatter(X_train, x='Longitude', y='Latitude', color='Time Bins', opacity=.7, marginal_y="rug", marginal_x="histogram")

In [0]:
px.scatter(X_train, x='Longitude', y='Latitude', color='Meter Id', opacity=.7, marginal_y="rug", marginal_x="histogram")

In [0]:
px.scatter_mapbox(df_new,lat='Latitude', lon='Longitude',color='fine_amount',opacity=.7)